In [15]:
import cv2
import numpy as np

# Load pre-trained face detector (use Haar cascades or a more advanced model if needed)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')  # Optional, for mouth

# Input and output video paths
input_video_path = 'myVideo.mp4'
output_video_path = 'output_with_silhouette.mp4'

# Initialize video capture and get properties
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Function to draw silhouette lines
def draw_silhouette_lines(frame, face_rect, eyes):
    x, y, w, h = face_rect
    
    # Draw vertical line for head (center of face)
    head_center_x = x + w // 2
    cv2.line(frame, (head_center_x, y), (head_center_x, y + h), (0, 255, 255), 2)
    
    # Draw vertical line for neck (estimate slightly below face)
    neck_y_start = y + h + 10  # start below the face
    neck_y_end = neck_y_start + 30  # length of neck line
    cv2.line(frame, (head_center_x, neck_y_start), (head_center_x, neck_y_end), (0, 255, 0), 2)
    
    # Draw horizontal line for shoulders
    shoulder_y = neck_y_end + 10
    shoulder_x_start = x - 20  # estimate shoulder width based on face width
    shoulder_x_end = x + w + 20
    cv2.line(frame, (shoulder_x_start, shoulder_y), (shoulder_x_end, shoulder_y), (255, 0, 255), 2)
    
    # Draw horizontal line for eyes if detected
    if len(eyes) > 0:
        eye_y = y + eyes[0][1] + eyes[0][3] // 2  # approximate eye line by taking the center of one eye
        cv2.line(frame, (x, eye_y), (x + w, eye_y), (0, 0, 255), 2)

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        # Detect eyes within face ROI
        roi_gray = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Draw silhouette lines on the detected parts
        draw_silhouette_lines(frame, (x, y, w, h), eyes)

    # Display the frame with silhouette in real-time
    cv2.imshow('Silhouette Overlay', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit early if needed

    # Write the processed frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. The output video is saved as:", output_video_path)


Processing complete. The output video is saved as: output_with_silhouette.mp4


In [24]:
import cv2
import numpy as np
import math

# Load pre-trained face and eye detectors
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_eye.xml')

# Input and output video paths
input_video_path = 'myvideo.mp4'
output_video_path = 'output_with_silhouette.mp4'

# Initialize video capture and get properties
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize VideoWriter to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps,
                      (frame_width, frame_height))

# Function to calculate angle between two points (eyes)


def calculate_angle(eye_left, eye_right):
    dx = eye_right[0] - eye_left[0]
    dy = eye_right[1] - eye_left[1]
    angle = math.degrees(math.atan2(dy, dx))
    return angle

# Function to draw silhouette lines for head (blue) and neck (red)


def draw_silhouette_lines(frame, face_rect, angle):
    x, y, w, h = face_rect

    # Calculate the center of the face for positioning lines
    face_center_x = x + w // 2
    face_center_y = y + h // 2

    # Length of the face (blue) and neck (red) lines
    face_line_length = int(h * 0.5)
    neck_line_length = int(h * 0.5)

    # Calculate the end point for the blue (head) line
    face_line_end_x = int(
        face_center_x + face_line_length * math.cos(math.radians(angle)))
    face_line_end_y = int(
        face_center_y + face_line_length * math.sin(math.radians(angle)))

    # Draw the blue line for the head, tilted with the head angle
    cv2.line(frame, (face_center_x, face_center_y), (face_line_end_x,
             face_line_end_y), (255, 0, 0), 2)  # Blue color for head line

    # Draw the red line for the neck, vertical below the end of the blue line
    neck_start_x = face_line_end_x
    neck_start_y = face_line_end_y
    neck_line_end_x = neck_start_x  # Keep the neck line vertically aligned
    neck_line_end_y = neck_start_y + neck_line_length

    cv2.line(frame, (neck_start_x, neck_start_y), (neck_line_end_x,
             neck_line_end_y), (0, 0, 255), 2)  # Red color for neck line


# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect face in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        # Detect eyes within face ROI to calculate head inclination
        roi_gray = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Ensure at least two eyes are detected to determine inclination
        if len(eyes) >= 2:
            # Get coordinates of the two detected eyes
            eye_left = (x + eyes[0][0] + eyes[0][2] // 2,
                        y + eyes[0][1] + eyes[0][3] // 2)
            eye_right = (x + eyes[1][0] + eyes[1][2] // 2,
                         y + eyes[1][1] + eyes[1][3] // 2)

            # Calculate the angle of the head tilt
            angle = calculate_angle(eye_left, eye_right)

            # Draw silhouette lines based on the calculated angle
            draw_silhouette_lines(frame, (x, y, w, h), angle)

    # Display the frame with silhouette in real-time
    cv2.imshow('Silhouette Overlay', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit early if needed

    # Write the processed frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print("Processing complete. The output video is saved as:", output_video_path)

Processing complete. The output video is saved as: output_with_silhouette.mp4
